# Week 2

## Hill Cipher


In [42]:
hill_transform <- function(m, message) {
  n <- nrow(m) # Assuming m is a square matrix
  # Pad message with 23
  pad_length <- n - length(message) %% n
  if (pad_length < n) {
    message <- c(message, rep(23, pad_length))
  }
  
  # Perform the transformation
  result <- numeric()
  for (i in seq(1, length(message), by = n)) {
    chunk <- matrix(message[i:(i+n-1)], nrow = n)
    transformed <- m %*% chunk
    result <- c(result, as.vector(transformed))
  }
  
  # Apply modulo 26 to the result
  result <- round(result) %% 26
  return(result)
}

### Check key and the inverse

$
Encipher: E = \begin{pmatrix}
6 & 24 & 1 \\
13 & 16 & 10 \\
20 & 17 & 15
\end{pmatrix}
Decipher: D = E^{-1} = \begin{pmatrix}
8 & 5 & 10 \\
21 & 8 & 21 \\
21 & 12 & 8
\end{pmatrix}
$

In [7]:
inverse_modulo <- function(E) {
  detE <- det(E)
  detE_inv_mod26 <- solve_mod(detE, 26)
  adjE <- solve(E) * detE
  invE_mod26 <- (detE_inv_mod26 * adjE) %% 26
  return(invE_mod26)
}

solve_mod <- function(a, m) {
  a_mod <- a %% m
  for (x in 1:(m-1)) {
    if ((a_mod * x) %% m == 1) {
      return(x)
    }
  }
  stop("Inverse does not exist")
}

In [40]:
E <- matrix(c(6, 24, 1, 13, 16, 10, 20, 17, 15), nrow=3, ncol=3, byrow=TRUE)
D <- inverse_modulo(E)
print(E)
print(D)

     [,1] [,2] [,3]
[1,]    6   24    1
[2,]   13   16   10
[3,]   20   17   15
     [,1] [,2] [,3]
[1,]    8    5   10
[2,]   21    8   21
[3,]   21   12    8


In [33]:
convert_string_to_integers <- function(plain_text) {
  # Convert to uppercase and remove non-letter characters
  filtered_text <- gsub("[^A-Z]", "", toupper(plain_text))
  # Convert letters to integers (0-25)
  int_vector <- as.integer(charToRaw(filtered_text)) - as.integer(charToRaw('A'))
  return(int_vector)
}

integers_to_string <- function(int_vector) {
  # Convert integers (0-25) to characters
  char_vector <-  as.raw(int_vector + as.integer(charToRaw('A')))
  # Convert raw values to characters and collapse into a string
  result_string <- paste(rawToChar(char_vector), collapse = "")
  return(result_string)
}

In [36]:
plain_text <- 'Hello, Meet me at the toga party!'
message <- convert_string_to_integers(plain_text)

print(convert_string_to_integers(plain_text))
print(integers_to_string(convert_string_to_integers(plain_text)))

 [1]  7  4 11 11 14 12  4  4 19 12  4  0 19 19  7  4 19 14  6  0 15  0 17 19 24
[1] "HELLOMEETMEATTHETOGAPARTY"


In [44]:
print(plain_text)
print(convert_string_to_integers(plain_text))
print(hill_transform(E, convert_string_to_integers(plain_text)))
cipher_text = integers_to_string(hill_transform(E, convert_string_to_integers(plain_text)))
print(cipher_text)
print(hill_transform(D, convert_string_to_integers(cipher_text)))
print(integers_to_string(hill_transform(D, convert_string_to_integers(cipher_text))))

[1] "Hello, Meet me at the toga party!"
 [1]  7  4 11 11 14 12  4  4 19 12  4  0 19 19  7  4 19 14  6  0 15  0 17 19 24
 [1] 19  5  9 24 19 14  9 20 17 12 12 22  5 23  2  0  2 15 25 20  7 11 20  2 17
[26]  0 20
[1] "TFJYTOJURMMWFXCACPZUHLUCRAU"
 [1]  7  4 11 11 14 12  4  4 19 12  4  0 19 19  7  4 19 14  6  0 15  0 17 19 24
[26] 23 23
[1] "HELLOMEETMEATTHETOGAPARTYXX"
